In [10]:
import pandas as pd
import glob

In [12]:
file_pattern = 'D:\\Tarun\\UTS\\Subjects\\ADV ML\\output\\*.csv'

In [13]:
file_list = glob.glob(file_pattern)

In [14]:
dfs = []

In [15]:
for file in file_list:
    df = pd.read_csv(file)
    dfs.append(df)

In [16]:
combined_df = pd.concat(dfs, ignore_index=True)

In [17]:
combined_df.shape

(1959980, 15)

In [9]:
combined_df.head(5)

,Unnamed: 0,store_id,item_id,wm_yr_wk,sell_price,date,d,event_name,event_type,id,dept_id,cat_id,state_id,sale,sales_revenue
0,0,CA_1,HOBBIES_1_001,11328,8.26,2013-08-08,923,Eid al-Fitr,Religious,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0,0.00
1,1,CA_1,HOBBIES_1_001,11332,8.26,2013-09-02,948,LaborDay,National,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,1,8.26
2,2,CA_1,HOBBIES_1_001,11338,8.26,2013-10-14,990,ColumbusDay,National,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0,0.00
3,3,CA_1,HOBBIES_1_001,11338,8.26,2013-10-15,991,EidAlAdha,Religious,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0,0.00
4,4,CA_1,HOBBIES_1_001,11340,8.26,2013-10-31,1007,Halloween,Cultural,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0,0.00


In [10]:
combined_df.dtypes

Unnamed: 0         int64
store_id          object
item_id           object
wm_yr_wk           int64
sell_price       float64
date              object
d                  int64
event_name        object
event_type        object
id                object
dept_id           object
cat_id            object
state_id          object
sale               int64
sales_revenue    float64
dtype: object

In [11]:
combined_df = combined_df[combined_df['d'] <= 1541]

In [12]:
combined_df.shape

(1959980, 15)

In [49]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [45]:
combined_df_temp = combined_df[combined_df['date'] > '2015-04-18']

In [47]:
combined_df_temp.shape

(0, 15)

In [18]:
df = combined_df[['store_id', 'item_id', 'date', 'sales_revenue']]

In [32]:
df.head(5)

,store_id,item_id,date,sales_revenue
0,CA_1,HOBBIES_1_001,2013-08-08,0.00
1,CA_1,HOBBIES_1_001,2013-09-02,8.26
2,CA_1,HOBBIES_1_001,2013-10-14,0.00
3,CA_1,HOBBIES_1_001,2013-10-15,0.00
4,CA_1,HOBBIES_1_001,2013-10-31,0.00


In [33]:
df = df.copy()
df['date'] = pd.to_datetime(df['date'])

In [11]:
df.isna().sum()

store_id         0
item_id          0
date             0
sales_revenue    0
dtype: int64

In [34]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [35]:
from sklearn.preprocessing import LabelEncoder
le_store_id = LabelEncoder()
le_item_id = LabelEncoder()

In [36]:
df['le_store_id'] = le_store_id.fit_transform(df['store_id'])
df['le_item_id'] = le_item_id.fit_transform(df['item_id'])

In [55]:
df.head(5)

,sales_revenue,year,month,day,le_store_id,le_item_id
0,0.00,2013,8,8,0,1437
1,8.26,2013,9,2,0,1437
2,0.00,2013,10,14,0,1437
3,0.00,2013,10,15,0,1437
4,0.00,2013,10,31,0,1437


In [38]:
encoding_map_store_id = dict(zip(df['store_id'], df['le_store_id']))

In [41]:
encoding_map_store_id['CA_3']

2

In [42]:
encoding_map_item_id = dict(zip(df['item_id'], df['le_item_id']))

In [43]:
encoding_map_item_id['HOBBIES_1_001']

1437

In [44]:
df = df.drop(['store_id', 'item_id', 'date'], axis='columns')

In [45]:
df_X_train = df.drop(['sales_revenue'], axis='columns')
df_Y_train = df.sales_revenue

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_X_train, df_Y_train, train_size=0.8)
X_test.head(5)

,year,month,day,le_store_id,le_item_id
975799,2012,11,11,3,2061
1898079,2014,10,13,9,1755
1397088,2014,4,22,6,477
575586,2015,4,11,2,257
1179115,2013,10,15,4,2902


In [47]:
from sklearn.linear_model import LinearRegression

In [48]:
model = LinearRegression()

In [49]:
model.fit(X_train, Y_train)

LinearRegression()

In [50]:
y_pred = model.predict(X_test)

In [51]:
from sklearn.metrics import mean_squared_error

In [52]:
mse = mean_squared_error(Y_test, y_pred)

In [53]:
mse

86.28152561033706

In [54]:
model.score(X_test, Y_test)

0.008101577213918576

In [42]:
from sklearn.model_selection import cross_val_score
print("cross val score - Linear Regression - ",
          cross_val_score(LinearRegression(), df_X_train, df_Y_train, cv = 5))

cross val score - Logistic Regression -  [ 0.00631753 -0.01102523 -0.04580186  0.00328414  0.00729696]


In [22]:
from sklearn.ensemble import RandomForestRegressor

In [23]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

In [24]:
rf_model.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [25]:
predictions = rf_model.predict(X_test)

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
mse = mean_squared_error(Y_test, predictions)

In [28]:
mse

51.14164121132908

In [29]:
rf_model.score(X_test, Y_test)

0.4640421804867597

In [30]:
import xgboost as xgb

In [31]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = 5, alpha = 10, n_estimators = 100)

In [32]:
xg_reg.fit(X_train, Y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [33]:
y_pred = xg_reg.predict(X_test)

In [34]:
mse = mean_squared_error(Y_test, y_pred)

In [35]:
mse

90.97493584751147

In [36]:
xg_reg.score(X_test, Y_test)

0.04659437803908506

In [37]:
from sklearn.neighbors import KNeighborsRegressor

In [38]:
knn_reg = KNeighborsRegressor(n_neighbors=5)

In [39]:
knn_reg.fit(X_train, Y_train)

KNeighborsRegressor()

In [40]:
y_pred = knn_reg.predict(X_test)

In [41]:
mse = mean_squared_error(Y_test, y_pred)

In [42]:
mse

80.34415691636649

In [43]:
knn_reg.score(X_test, Y_test)

0.15800357337796267

In [55]:
from sklearn.ensemble import IsolationForest

In [56]:
clf = IsolationForest(contamination=0.05)

In [57]:
y_pred = clf.fit_predict(df)

D:\Tarun\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [58]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [59]:
outlier_mask = y_pred == -1

In [60]:
outlier_mask

array([False, False, False, ..., False, False, False])

In [63]:
df_no_outliers = df[~outlier_mask]

In [64]:
df_no_outliers.shape

(1861987, 6)

In [66]:
df_X_train = df_no_outliers.drop(['sales_revenue'], axis='columns')
df_Y_train = df_no_outliers.sales_revenue

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_X_train, df_Y_train, train_size=0.8)
X_test.head(5)

,year,month,day,le_store_id,le_item_id
1203220,2013,5,12,4,2981
1516205,2014,5,26,6,2491
480381,2014,7,29,1,2362
1536875,2014,5,11,7,691
487046,2011,5,31,1,2513


In [70]:
from sklearn.ensemble import RandomForestRegressor

In [71]:
rf_model_outliers = RandomForestRegressor(n_estimators=100, random_state=42) 

In [72]:
rf_model_outliers.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [73]:
predictions = rf_model_outliers.predict(X_test)

In [75]:
mse = mean_squared_error(Y_test, predictions)

In [76]:
mse

17.120664679889906

In [77]:
rf_model_outliers.score(X_test, Y_test)

0.2410296845193477

In [9]:
X_test.tail(1)

NameError: name 'X_test' is not defined

In [83]:
# pip install fastapi

Note: you may need to restart the kernel to use updated packages.


In [85]:
# pip install "uvicorn[standard]"

In [87]:
# pip install uvicorn

Note: you may need to restart the kernel to use updated packages.


In [1]:
from fastapi import FastAPI

In [2]:
app = FastAPI()

In [3]:
@app.get("/")
def read_root():
    return {"Hello": "World"}

In [22]:
import datetime

In [58]:
@app.get("/predict_revenue")
def predict_revenue(
    store_id: str,
    item_id: str,
    date: str
):
    datetime_object = datetime.datetime.strptime(date, '%Y-%m-%d')
    
    pred = rf_model.predict([[
        datetime_object.year, 
        datetime_object.month, 
        datetime_object.day, 
        encoding_map_store_id[store_id], 
        encoding_map_item_id[item_id]]])
    return pred

In [63]:
print(predict_revenue('WI_3', 'HOUSEHOLD_1_201', '2015-04-12'))

[2.27132452]


D:\Tarun\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
